# Home Kredit Default Risk
####    By Arif Putera Wijaya
    
    Task : menggunakan berbagai macam metode statistik dan Machine Learning untuk membuat prediksi skor kredit, ntuk membuka potensi maksimal dari data kami. Dengan melakukannya, kita dapat memastikan pelanggan yang mampu melakukan pelunasan tidak ditolak ketika melakukan pengajuan pinjaman, dan pinjaman datap diberikan dengan principal, maturity, dan repayment calendar yang akan memotivsi pelanggan untuk sukses.menggunakan setidaknya 2 model Machine Learning dimana salah satunya adalah Logistic Regression

#### Import Library & Load data

In [1]:
import pandas as pd 
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

import sys
import warnings
warnings.filterwarnings("ignore")

pd.pandas.set_option('display.max_columns', None)

In [2]:
application_train = pd.read_csv('cleandata.csv')

In [3]:
application_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,CODE_GENDER_F,CODE_GENDER_M,CODE_GENDER_XNA,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,NAME_INCOME_TYPE_Businessman,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_EDUCATION_TYPE_Academic degree,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Unknown,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_Co-op apartment,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,OCCUPATION_TYPE_Accountants,OCCUPATION_TYPE_Cleaning staff,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_HR staff,OCCUPATION_TYPE_High skill tech staff,OCCUPATION_TYPE_IT staff,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_Private service staff,OCCUPATION_TYPE_Realty agents,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Secretaries,OCCUPATION_TYPE_Security staff,OCCUPATION_TYPE_Waiters/barmen staff,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,ORGANIZATION_TYPE_Advertising,ORGANIZATION_TYPE_Agriculture,ORGANIZATION_TYPE_Bank,ORGANIZATION_TYPE_Business Entity Type 1,ORGANIZATION_TYPE_Business Entity Type 2,ORGANIZATION_TYPE_Business Entity Type 3,ORGANIZATION_TYPE_Cleaning,ORGANIZATION_TYPE_Construction,ORGANIZATION_TYPE_Culture,ORGANIZATION_TYPE_Electricity,ORGANIZATION_TYPE_Emergency,ORGANIZATION_TYPE_Government,ORGANIZATION_TYPE_Hotel,ORGANIZATION_TYPE_Housing,ORGANIZATION_TYPE_Industry: type 1,ORGANIZATION_TYPE_Industry: type 10,ORGANIZATION_TYPE_Industry: type 11,ORGANIZATION_TYPE_Industry: type 12,ORGANIZATION_TYPE_Industry: type 13,ORGANIZATION_TYPE_Industry: type 2,ORGANIZATION_TYPE_Industry: type 3,ORGANIZATION_TYPE_Industry: type 4,ORGANIZATION_TYPE_Industry: type 5,ORGANIZATION_TYPE_Industry: type 6,ORGANIZATION_TYPE_Industry: ty

#### Outlier_fraction

In [4]:
Fraud = application_train[application_train['TARGET']==1]
Valid = application_train[application_train['TARGET']==0]

outlier_fraction = (len(Fraud)/float(len(Valid)))*100

print('outlier_fraction untuk seluruh dataset: {:.3f}%'.format(outlier_fraction))
print("Fraud Cases: {}".format(len(Fraud)))
print("Valid Cases : {}".format(len(Valid)))

outlier_fraction untuk seluruh dataset: 8.782%
Fraud Cases: 24825
Valid Cases : 282686


    Dalam script tersebut, application_train adalah nama dataset yang digunakan. Variabel Fraud menyimpan data transaksi yang dianggap sebagai Client Gagal bayar (dengan nilai kolom 'TARGET' sama dengan 1), sedangkan variabel Valid menyimpan data transaksi yang dianggap valid (dengan nilai kolom 'TARGET' sama dengan 0).

    Rasio outlier_fraction dihitung sebagai jumlah transaksi Gagal bayar dibagi dengan jumlah transaksi valid. Hasilnya mencerminkan seberapa besar proporsi transaksi gagal bayar dalam dataset. Akhirnya, script menampilkan nilai outlier_fraction dan jumlah kasus.


#### Split Data

In [5]:
from sklearn.model_selection import train_test_split

columns = application_train.columns.tolist()
columns = [c for c in columns if c not in ["TARGET"]]
target = "TARGET"

state = np.random.RandomState(42)
X = application_train[columns]
y = application_train[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

print("Bentuk Subset Pelatihan:", X_train.shape, y_train.shape)
print("Bentuk Subset Pengujian:", X_test.shape, y_test.shape)

Bentuk Subset Pelatihan: (215257, 210) (215257,)
Bentuk Subset Pengujian: (92254, 210) (92254,)


### Feature Selection


In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

In [7]:
sel = SelectFromModel(RandomForestClassifier(n_estimators = 100, random_state=state))
sel.fit(X_train, y_train)

SelectFromModel(estimator=RandomForestClassifier(random_state=RandomState(MT19937) at 0x28B4BB14B40))

In [8]:
sel.get_support()

array([ True, False, False,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False, False, False, False,
       False,  True,  True,  True, False, False, False, False, False,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True, False,  True, False,  True,
        True, False,  True, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
        True,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [9]:
selected_feat= X_train.columns[(sel.get_support())].tolist()
len(selected_feat)

39

In [41]:
selected_feat

['SK_ID_CURR',
 'FLAG_OWN_REALTY',
 'CNT_CHILDREN',
 'AMT_INCOME_TOTAL',
 'AMT_CREDIT',
 'AMT_ANNUITY',
 'REGION_POPULATION_RELATIVE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_REGISTRATION',
 'DAYS_ID_PUBLISH',
 'OWN_CAR_AGE',
 'CNT_FAM_MEMBERS',
 'REGION_RATING_CLIENT',
 'HOUR_APPR_PROCESS_START',
 'EXT_SOURCE_1',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'APARTMENTS_AVG',
 'BASEMENTAREA_AVG',
 'YEARS_BEGINEXPLUATATION_AVG',
 'YEARS_BUILD_AVG',
 'COMMONAREA_AVG',
 'ENTRANCES_AVG',
 'FLOORSMAX_AVG',
 'FLOORSMIN_AVG',
 'LANDAREA_AVG',
 'NONLIVINGAREA_AVG',
 'OBS_30_CNT_SOCIAL_CIRCLE',
 'DEF_30_CNT_SOCIAL_CIRCLE',
 'DAYS_LAST_PHONE_CHANGE',
 'AMT_REQ_CREDIT_BUREAU_MON',
 'AMT_REQ_CREDIT_BUREAU_QRT',
 'AMT_REQ_CREDIT_BUREAU_YEAR',
 'NAME_FAMILY_STATUS_Married',
 'CREDIT_INCOME_PERCENT',
 'ANNUITY_INCOME_PERCENT',
 'CREDIT_TERM',
 'DAYS_EMPLOYED_PERCENT']

In [10]:
app_X_train = X_train.copy() #Taking a copy before dropping 
app_X_test = X_test.copy()

In [11]:
X_train = X_train[selected_feat]
X_test = X_test[selected_feat]

In [12]:

print (X_train.shape, X_test.shape)

(215257, 39) (92254, 39)


### Logistic Regression

In [13]:
from sklearn.linear_model import LogisticRegression

logistic_regressor = LogisticRegression(C = 2)
logistic_regressor.fit(X_train,y_train)

LogisticRegression(C=2)

In [14]:
y_pred = logistic_regressor.predict(X_test)

In [15]:
from sklearn.metrics import classification_report, accuracy_score, f1_score, roc_auc_score
from sklearn.metrics import confusion_matrix

n_errors = (y_pred != y_test).sum()

print("{}: {}".format("Logistic Regression errors", n_errors))

print("Accuracy Score:")
print(accuracy_score(y_test, y_pred))

print("\nConfusion matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("ROC AUC score is: ", roc_auc_score(y_test, y_pred))

Logistic Regression errors: 7425
Accuracy Score:
0.9195156849567498

Confusion matrix:
[[84829     0]
 [ 7425     0]]

Classification Report:
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     84829
           1       0.00      0.00      0.00      7425

    accuracy                           0.92     92254
   macro avg       0.46      0.50      0.48     92254
weighted avg       0.85      0.92      0.88     92254

ROC AUC score is:  0.5


###### Jumlah Kesalahan (Errors): 
Terdapat 7,313 kesalahan pada model Logistic Regression.

###### Akurasi (Accuracy): 
Akurasi model sebesar 92.07% terlihat tinggi. Namun, perlu dicatat bahwa dalam kasus ini, akurasi mungkin tidak menjadi metrik yang paling informatif karena distribusi kelas sangat tidak seimbang (class imbalance). Kelas 1 (TARGET = 1) memiliki jumlah sampel yang jauh lebih sedikit daripada kelas 0.

###### Confusion Matrix: 
Confusion matrix menunjukkan bahwa model sepenuhnya memprediksi kelas 0 (tidak ada kredit macet) dan tidak dapat mengidentifikasi kelas 1 (kredit macet). Ini terlihat dari seluruh nilai True Positive (7313) yang dianggap sebagai False Negative, sehingga Recall dan Precision untuk kelas 1 menjadi nol.

###### Classification Report: 
Precision, Recall, dan F1-score untuk kelas 1 semuanya nol, menunjukkan bahwa model tidak dapat mengklasifikasikan dengan baik kasus positif (kredit macet).

###### ROC AUC Score: 
ROC AUC score sebesar 0.5 menunjukkan bahwa model tidak dapat membedakan antara kelas positif dan kelas negatif, yang juga terlihat dari kurva ROC yang merupakan garis diagonal.

###### Kesimpulannya 
model ini tidak efektif dalam mengklasifikasikan kasus positif (kredit macet), dan perlu dilakukan perbaikan atau peningkatan model.

## SMOTETomek (Synthetic Minority Over-sampling Technique - Tomek links)
Metode yang digunakan untuk menangani ketidakseimbangan kelas (class imbalance) dalam dataset. Metode ini merupakan pendekatan hibrida yang menggabungkan dua teknik, yaitu SMOTE (Synthetic Minority Over-sampling Technique) dan Tomek links.

Penggunaan SMOTETomek sebagai metode hibrida memanfaatkan kelebihan kedua teknik yang digunakan, yaitu oversampling untuk kelas minoritas (SMOTE) dan pengurangan instance redundan (Tomek links). Dengan demikian, SMOTETomek berusaha untuk mencapai keseimbangan yang lebih baik antara kelas dalam dataset, menghasilkan model yang lebih baik dan lebih umumnya dapat diterima pada masalah ketidakseimbangan kelas.


In [16]:
#!pip install -U imbalanced-learn

In [17]:
# from imblearn.combine import SMOTETomek
# from imblearn.under_sampling import NearMiss

In [18]:
# smk = SMOTETomek(random_state=42)
# X_res, y_res = smk.fit_resample(X_train, y_train)

In [19]:
# X_res.shape,y_res.shape

In [20]:
# from collections import Counter
# print('Original dataset shape {}'.format(Counter(y_train)))
# print('Resampled dataset shape {}'.format(Counter(y_res)))

#### SMOTETomek - Logistic Regression

In [21]:
# logistic_regressor.fit(X_res,y_res)

In [22]:
# y_pred = logistic_regressor.predict(X_test)

In [23]:
# n_errors = (y_pred != y_test).sum()

# print("{}: {}".format("Logistic Regression errors", n_errors))

# print("Accuracy Score:")
# print(accuracy_score(y_test, y_pred))

# print("\nConfusion matrix:")
# print(confusion_matrix(y_test, y_pred))

# print("\nClassification Report:")
# print(classification_report(y_test, y_pred))

# print("ROC AUC score is: ", roc_auc_score(y_test, y_pred))

## RandomOverSampler 
Merupakan  salah satu metode oversampling yang digunakan untuk menangani ketidakseimbangan kelas dalam dataset. Metode ini bekerja dengan cara menambahkan salinan acak dari sampel kelas minoritas (kelas yang kurang banyak representasinya) sehingga jumlah sampel pada kedua kelas menjadi seimbang atau mendekati seimbang.

In [24]:
#!pip install -U imbalanced-learn

In [25]:
from imblearn.over_sampling import RandomOverSampler
os =  RandomOverSampler(sampling_strategy=1)
X_train_res, y_train_res = os.fit_resample(X_train, y_train)
X_train_res.shape,y_train_res.shape

((395714, 39), (395714,))

In [26]:
from collections import Counter
print('Original dataset shape {}'.format(Counter(y_train)))
print('Resampled dataset shape {}'.format(Counter(y_train_res)))

Original dataset shape Counter({0: 197857, 1: 17400})
Resampled dataset shape Counter({0: 197857, 1: 197857})


#### RandomOverSampler - Logistic Regression

In [27]:
logistic_regressor.fit(X_train_res,y_train_res)

LogisticRegression(C=2)

In [28]:
y_pred = logistic_regressor.predict(X_test)

In [29]:
n_errors = (y_pred != y_test).sum()

print("{}: {}".format("Logistic Regression errors", n_errors))

print("Accuracy Score:")
print(accuracy_score(y_test, y_pred))

print("\nConfusion matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("ROC AUC score is: ", roc_auc_score(y_test, y_pred))

Logistic Regression errors: 39145
Accuracy Score:
0.5756823552366293

Confusion matrix:
[[49036 35793]
 [ 3352  4073]]

Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.58      0.71     84829
           1       0.10      0.55      0.17      7425

    accuracy                           0.58     92254
   macro avg       0.52      0.56      0.44     92254
weighted avg       0.87      0.58      0.67     92254

ROC AUC score is:  0.563304610467491


#### RandomOverSampler - Random forest

In [30]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(n_estimators = 100, random_state = state, verbose = 1, n_jobs = -1)

In [31]:
random_forest.fit(X_train_res,y_train_res)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   49.7s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  2.1min finished


RandomForestClassifier(n_jobs=-1,
                       random_state=RandomState(MT19937) at 0x28B4BB14B40,
                       verbose=1)

In [32]:
y_pred = random_forest.predict(X_test)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.8s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    2.0s finished


In [33]:
n_errors = (y_pred != y_test).sum()

print("{}: {}".format("Random Forest errors",n_errors))
print("\nAccuracy Score :")
print(accuracy_score(y_test,y_pred))

print("\nConfusion matrix :")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report :")
print(classification_report(y_test,y_pred))

print("\nROC AUC score is: ",roc_auc_score(y_test,y_pred))

Random Forest errors: 7532

Accuracy Score :
0.9183558436490559

Confusion matrix :
[[84495   334]
 [ 7198   227]]

Classification Report :
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     84829
           1       0.40      0.03      0.06      7425

    accuracy                           0.92     92254
   macro avg       0.66      0.51      0.51     92254
weighted avg       0.88      0.92      0.88     92254


ROC AUC score is:  0.5133175289103097


######  Kesalahan (Errors):
Model Random Forest menghasilkan 7,532 kesalahan klasifikasi.

###### Akurasi (Accuracy):
Akurasi model Random Forest adalah sekitar 91.84%. Model ini memiliki tingkat akurasi yang tinggi, tetapi kita perlu melihat lebih detail pada performa kelas minoritas (kelas 1 - fraud).
Confusion Matrix:

###### Confusion matrix 
menunjukkan bahwa model cenderung mengklasifikasikan sebagian besar instances sebagai kelas mayoritas (kelas 0 - non-fraud). Jumlah False Positives (334) relatif rendah dibandingkan dengan model XGBoost sebelumnya.

###### Classification Report:
Precision untuk kelas 1 (fraud) adalah 0.40%, menunjukkan bahwa dari prediksi positif yang dibuat oleh model, hanya sebagian kecil yang benar-benar merupakan kasus fraud.
Recall untuk kelas 1 sangat rendah (0.03%), menunjukkan bahwa model gagal mengidentifikasi sebagian besar kasus fraud yang sebenarnya.
F1-score untuk kelas 1 juga rendah, menunjukkan kesulitan model dalam menggabungkan precision dan recall.

###### ROC AUC Score:
ROC AUC score sekitar 0.51 menunjukkan kemampuan model untuk membedakan antara kelas positif dan negatif sangat rendah. Skor ini menunjukkan bahwa Random Forest tidak efektif dalam memprediksi kasus fraud.

###### Analisis Keseluruhan:
Meskipun memiliki akurasi yang tinggi, Random Forest menghadapi tantangan dalam mengidentifikasi dan memprediksi kasus fraud (kelas 1). Model ini lebih cenderung menghasilkan False Negatives (misclassifying fraud cases as non-fraud) dibandingkan dengan False Positives.

Dibandingkan dengan XGBoost, Random Forest memiliki performa yang lebih rendah dalam hal kemampuan membedakan kelas positif dan negatif (ROC AUC score).

Perlu pertimbangan lebih lanjut untuk meningkatkan kemampuan model dalam mendeteksi kasus fraud, seperti eksplorasi lebih lanjut terhadap fitur-fitur atau penyetelan hyperparameter.

## XGBoost
Extreme Gradient Boosting, adalah sebuah algoritma machine learning yang termasuk dalam kategori pohon keputusan berbasis ensemble. Ensemble learning adalah paradigma di mana beberapa model yang relatif lemah digabungkan untuk membentuk model yang lebih kuat dan lebih kompleks. XGBoost secara khusus dikembangkan untuk meningkatkan kinerja dan kestabilan model ensemble.

In [38]:
X_train_res = pd.DataFrame(data=X_train_res, columns=selected_feat)

In [39]:
X_train_res.shape

(395714, 39)

In [ ]:
#!pip install xgboost

In [36]:
from xgboost import XGBClassifier
xgb_classifier = XGBClassifier(n_estimators=100,max_depth=5)

In [37]:
xgb_classifier.fit(X_train_res,y_train_res)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [40]:
y_pred = xgb_classifier.predict(X_test)

n_errors = (y_pred != y_test).sum()
print("{}: {}".format("Extreme Gradient Boost errors",n_errors))

print("\nAccuracy Score :")
print(accuracy_score(y_test,y_pred))

print("\nConfusion matrix :")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report :")
print(classification_report(y_test,y_pred))

print("\nROC AUC score is: ",roc_auc_score(y_test,y_pred))

Extreme Gradient Boost errors: 25491

Accuracy Score :
0.7236867778090923

Confusion matrix :
[[62014 22815]
 [ 2676  4749]]

Classification Report :
              precision    recall  f1-score   support

           0       0.96      0.73      0.83     84829
           1       0.17      0.64      0.27      7425

    accuracy                           0.72     92254
   macro avg       0.57      0.69      0.55     92254
weighted avg       0.90      0.72      0.78     92254


ROC AUC score is:  0.6853215625350154


###### Jumlah Kesalahan (Errors):
Model XGBoost menghasilkan lebih sedikit kesalahan dibandingkan dengan model Regresi Logistik sebelumnya, yang mengalami 25,491 kesalahan klasifikasi.
Akurasi (Accuracy):

###### Akurasi model XGBoost 
adalah sekitar 72.37%, meningkat dari akurasi sebelumnya dengan Regresi Logistik.
Confusion Matrix:

###### Confusion matrix 
menunjukkan bahwa model cenderung membuat kesalahan dalam mengklasifikasikan kelas 1 (fraud), yang dapat dilihat dari False Negatives (2676). Namun, jumlah False Positives (22815) juga cukup tinggi.
Classification Report:

###### Precision 
untuk kelas 1 (fraud) masih rendah (17%), menunjukkan bahwa dari prediksi positif yang dibuat oleh model, hanya sebagian kecil yang benar-benar merupakan kasus fraud.

###### Recall 
untuk kelas 1 meningkat menjadi 64%, yang menunjukkan bahwa model dapat mengidentifikasi sebagian besar kasus fraud yang sebenarnya.

###### F1-score 
untuk kelas 1 juga meningkat, tetapi masih tergolong rendah, menunjukkan bahwa model belum sempurna dalam menggabungkan precision dan recall.

Meskipun akurasi meningkat, kita perlu memperhatikan trade-off antara precision dan recall.

###### ROC AUC Score:
ROC AUC score sekitar 0.685 menunjukkan kemampuan model untuk membedakan antara kelas positif dan negatif. Skor ini menunjukkan peningkatan dari model sebelumnya, tetapi masih perlu diperhatikan untuk meningkatkannya lebih lanjut.

###### Analisis Keseluruhan:
Model XGBoost memberikan peningkatan yang signifikan dibandingkan dengan Regresi Logistik, terutama dalam hal akurasi dan kemampuan membedakan kelas positif dan negatif.

Meskipun peningkatan, performa model masih bisa ditingkatkan, terutama dalam mengatasi False Positives dan meningkatkan precision pada kelas 1.

Penting untuk mempertimbangkan strategi penanganan ketidakseimbangan kelas dan penyetelan parameter lebih lanjut untuk meningkatkan performa model.